Чернышова Дана Кирилловна

Task 6

In [2]:
import numpy as np
import heapq
import time
from collections import defaultdict
import random
import random
import heapq

In [3]:
def generate_adjacency_matrix(vertices, edges, max_weight=20):
    matrix = np.zeros((vertices, vertices), dtype=int)
    edge_count = 0

    while edge_count < edges:
        u = random.randint(0, vertices - 1)
        v = random.randint(0, vertices - 1)
        
        if u != v and matrix[u, v] == 0:  # Ensure no self-loops and no duplicate edges
            weight = random.randint(1, max_weight)
            matrix[u, v] = weight
            matrix[v, u] = weight
            edge_count += 1

    return matrix

# Dijkstra's Algorithm
def dijkstra(graph, start):
    distances = [float('inf')] * len(graph)
    distances[start] = 0
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor in range(len(graph)):
            if graph[current_vertex][neighbor] != 0:
                distance = current_distance + graph[current_vertex][neighbor]

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))

    return distances

# Bellman-Ford Algorithm
def bellman_ford(graph, start):
    distances = [float('inf')] * len(graph)
    distances[start] = 0

    for _ in range(len(graph) - 1):
        for u in range(len(graph)):
            for v in range(len(graph)):
                if graph[u][v] != 0:
                    if distances[u] + graph[u][v] < distances[v]:
                        distances[v] = distances[u] + graph[u][v]

    # Check for negative-weight cycles
    for u in range(len(graph)):
        for v in range(len(graph)):
            if graph[u][v] != 0:
                if distances[u] + graph[u][v] < distances[v]:
                    raise ValueError("Graph contains a negative-weight cycle")

    return distances

In [4]:
vertices = 100
edges = 500
adjacency_matrix = generate_adjacency_matrix(vertices, edges)

start_vertex = random.randint(0, vertices - 1)

dijkstra_times = []
bellman_ford_times = []

for _ in range(10):
    # Measure time for Dijkstra's
    start_time = time.time()
    dijkstra(adjacency_matrix, start_vertex)
    dijkstra_times.append(time.time() - start_time)

    # Measure time for Bellman-Ford
    start_time = time.time()
    bellman_ford(adjacency_matrix, start_vertex)
    bellman_ford_times.append(time.time() - start_time)

avg_dijkstra_time = sum(dijkstra_times) / len(dijkstra_times)
avg_bellman_ford_time = sum(bellman_ford_times) / len(bellman_ford_times)

print(f"Average time for Dijkstra's Algorithm: {avg_dijkstra_time:.6f} seconds")
print(f"Average time for Bellman-Ford Algorithm: {avg_bellman_ford_time:.6f} seconds")

Average time for Dijkstra's Algorithm: 0.001323 seconds
Average time for Bellman-Ford Algorithm: 0.112204 seconds


In [5]:
# Directions for movement in the grid (4 possible directions: up, down, left, right)
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# A* Algorithm
def a_star(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    
    def heuristic(a, b):
        # Manhattan distance heuristic
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    # Priority queue for A* (stores tuples of (f, g, current_position))
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start))
    
    # Cost from start to current node (g value)
    g_costs = {start: 0}
    
    # Dictionary to track the path
    came_from = {}
    
    while open_list:
        _, current_g, current = heapq.heappop(open_list)
        
        if current == goal:
            # Reconstruct the path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path
        
        for direction in DIRECTIONS:
            neighbor = (current[0] + direction[0], current[1] + direction[1])
            
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] == 0:
                tentative_g = current_g + 1
                if neighbor not in g_costs or tentative_g < g_costs[neighbor]:
                    g_costs[neighbor] = tentative_g
                    f = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f, tentative_g, neighbor))
                    came_from[neighbor] = current
    
    return None  # If no path is found

# Generate a 10x20 grid with obstacles
def generate_grid(rows=10, cols=20, num_obstacles=40):
    grid = [[0] * cols for _ in range(rows)]  # 0 represents an empty cell
    
    # Place obstacles (represented by 1)
    obstacle_cells = random.sample(range(rows * cols), num_obstacles)
    for idx in obstacle_cells:
        r, c = divmod(idx, cols)
        grid[r][c] = 1  # Mark this cell as an obstacle
    
    return grid

# Find a random non-obstacle cell
def find_random_non_obstacle(grid):
    while True:
        r = random.randint(0, len(grid) - 1)
        c = random.randint(0, len(grid[0]) - 1)
        if grid[r][c] == 0:
            return (r, c)

In [6]:
rows, cols = 10, 20
num_obstacles = 40

path_lengths = []
times = []

for _ in range(5):  # Repeat the experiment 5 times
    grid = generate_grid(rows, cols, num_obstacles)
    
    # Pick two random non-obstacle cells
    start = find_random_non_obstacle(grid)
    goal = find_random_non_obstacle(grid)
    
    # Measure time for A* algorithm
    start_time = time.time()
    path = a_star(grid, start, goal)
    end_time = time.time()
    
    times.append(end_time - start_time)
    
    if path is None:
        path_lengths.append(0)  # No path found
    else:
        path_lengths.append(len(path))  # Path length
    
    # Optional: Print the grid and the path for each run
    print(f"Start: {start}, Goal: {goal}")
    print("Grid:")
    for row in grid:
        print(" ".join(str(cell) for cell in row))
    print(f"Path: {path}")
    print()

avg_time = sum(times) / len(times)
avg_path_length = sum(path_lengths) / len(path_lengths)

print(f"Average time for A* algorithm: {avg_time:.6f} seconds")
print(f"Average path length: {avg_path_length:.2f} cells")

Start: (7, 11), Goal: (2, 1)
Grid:
0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1
0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0
0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
Path: [(7, 11), (7, 12), (6, 12), (5, 12), (5, 11), (5, 10), (5, 9), (4, 9), (3, 9), (2, 9), (2, 8), (2, 7), (2, 6), (2, 5), (2, 4), (2, 3), (2, 2), (2, 1)]

Start: (1, 15), Goal: (1, 12)
Grid:
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0
0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0
1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0
0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0
0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1
1 0 0 0 0 0

Heuristic (Manhattan Distance) -> The heuristic function estimates the cost to reach the goal from any given cell. In this case, Manhattan distance is used, which is the sum of the absolute differences between the current cell's coordinates and the goal cell's coordinates. This heuristic is appropriate for a grid with orthogonal movement (up, down, left, right).

Priority Queue (for A Algorithm)* ->A priority queue is used to select the next cell to explore in the A* algorithm, based on the lowest f-cost, where f=g+hf=g+h:
        g: The cost to reach the current node (start to current).
        h: The heuristic cost to reach the goal (usually Manhattan distance).

Cost Tracking (g-costs and Came-from Mapping) -> Keeps track of the actual cost to reach each cell. The value is updated whenever a cheaper path to a cell is found. came_from: This helps to reconstruct the path after the goal has been found. It stores the parent (previous) cell of each cell, effectively storing the path backwards from the goal to the start.